# 11月21日　ゼミ

In [1]:
using LaTeXStrings
include("../display/display-crisp-matrix.jl")
include("../display/display-interval-vector.jl")
include("../entani/partial-incorporarion.jl")
include("../ttimes/optimal-value.jl")
include("../ttimes/t-perfect.jl")

solvetPerfectIncorporationLP (generic function with 1 method)

In [2]:
A₁ = [
    1 2 3 4
    1/2 1 2 3
    1/3 1/2 1 2
    1/4 1/3 1/2 1
]

A₂ = [
    1 3 3 4
    1/3 1 3 3
    1/3 1/3 1 4
    1/4 1/3 1/4 1
]

A₃ = [
    1 1 2 2
    1 1 3 1
    1/2 1/3 1 3
    1/2 1 1/3 1
]

4×4 Matrix{Float64}:
 1.0  1.0       2.0       2.0
 1.0  1.0       3.0       1.0
 0.5  0.333333  1.0       3.0
 0.5  1.0       0.333333  1.0

# partial incorporation approachの実装

$$
    \begin{align}
        \begin{split}
            \text{minimize} ~~ & ~~ \sum_{i\in N} (w_{i}^{U} - w_{i}^{L}) \\
            \text{subject to}  ~~ & ~~ \sum_{i\in N} (w_{ki}^{U} - w_{ki}^{L}) \leq \hat{d_{k}} , k\in M \\
            ~~ & ~~ w_{ki}^{L}\leq a_{kij}w_{kj}^{R},\, a_{kij}w_{kj}^{L}\leq w_{ki}^{U},\, i,j,\in N(i<j) , k\in M\\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{R} + w_{ki}^{L}\geq 1, k\in M, i\in N, \\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{L} + w_{ki}^{U}\leq 1, k\in M, i\in N, \\
            ~~ & ~~ \sum_{i\in N} w_{ki} = 1, k\in M, \\
            ~~ & ~~ \epsilon \leq w_{i}^{L}\leq w_{ki}\leq w_{ki}^{U}, \epsilon \leq w_{ki}^{L}\leq w_{ki}\leq w_{ki}^{U}, k\in M, i\in N.
        \end{split}
    \end{align}
$$


In [10]:
solution = solvePartialIncorporationLP([A₁, A₂, A₃])
W = solution.W
display(L"W = %$(intervalVectorLaTeXString(W))")

L"$W = \begin{pmatrix} \left[ 0.375, 0.571 \right] \\ \left[ 0.214, 0.25 \right] \\ \left[ 0.167, 0.167 \right] \\ \left[ 0.048, 0.208 \right] \end{pmatrix}$"

In [11]:
d = solution.optimalValue

0.39285714285714335

# 区間AHP問題に対して重みをp倍する
論文の各個人の区間重みを求める問題
$$
    \begin{align}
        \begin{split}
            \text{minimize} ~~ & ~~ \sum_{i\in N} (w_{ki}^{U} - w_{ki}^{L}) \\
            \text{subject to} ~~ & ~~ w_{ki}^{L}\leq a_{kij}w_{kj}^{R},\, a_{kij}w_{kj}^{L}\leq w_{ki}^{U},\, i,j,\in N(i<j) \\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{R} + w_{ki}^{L}\geq 1, i\in N, \\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{L} + w_{ki}^{U}\leq 1, i\in N, \\
            ~~ & ~~ w_{ki}^{U}\geq w_{ki}^{L}\geq \epsilon, i\in N. 
        \end{split}
    \end{align}
$$

この問題によって、各個人の区間重みが求まる

曖昧性があることを利用して、うまく意見を統合する目的で、この区間を広げたり狭めたりしてもいいんじゃないか？

重みをp倍した問題を考える。pは以下のように定義する
$$
    \begin{align}
        \begin{split}
            
            ~~ & ~~ p = \frac{1}{\sum_{i\in N} \frac{(w_{ki}^{U} + w_{ki}^{L})}{2}} \\
            ~~ & ~~ v_{ki}^{R} = pw_{ki}^{U} , v_{ki}^{L} = pw_{ki}^{L} \\
        \end{split}
    \end{align}
$$

つまり下のような問題
$$
    \begin{align}
        \begin{split}
            \text{minimize} ~~ & ~~ \frac{\sum_{i\in N} (w_{ki}^{U} - w_{ki}^{L})}{\sum_{i\in N} \frac{(w_{ki}^{U} + w_{ki}^{L})}{2}} \\
            \text{subject to} ~~ & ~~ ...
        \end{split}
    \end{align}
$$

上の$v$を用いて書くと下のような問題

$$
    \begin{align}
        \begin{split}
            \text{minimize} ~~ & ~~ \sum_{i\in N} (v_{ki}^{R} - v_{ki}^{L}) \\
            \text{subject to} ~~ & ~~ v_{ki}^{L}\leq a_{kij}v_{kj}^{R},\, a_{kij}v_{kj}^{L}\leq v_{ki}^{R},\, i,j,\in N(i<j) \\
            ~~ & ~~ \sum_{j\in N\setminus i} v_{kj}^{R} + v_{ki}^{L}\geq p, i\in N, \\
            ~~ & ~~ \sum_{j\in N\setminus i} v_{kj}^{L} + v_{ki}^{R}\leq p, i\in N, \\
            ~~ & ~~ v_{ki}^{R}\geq v_{ki}^{L}\geq \epsilon, i\in N. 
        \end{split}
    \end{align}
$$

これが下の問題と同値である

$$
    \begin{align}
        \begin{split}
            \text{minimize} ~~ & ~~ \sum_{i\in N} (w_{ki}^{U} - w_{ki}^{L}) \\
            \text{subject to} ~~ & ~~ w_{ki}^{L}\leq a_{kij}w_{kj}^{R},\, a_{kij}w_{kj}^{L}\leq w_{ki}^{U},\, i,j,\in N(i<j) \\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{R} + w_{ki}^{L}\geq 1, i\in N, \\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{L} + w_{ki}^{U}\leq 1, i\in N, \\
            ~~ & ~~ \sum_{i\in N} (w_{ki}^{U} + w_{ki}^{L}) = 2 \\
            ~~ & ~~ w_{ki}^{U}\geq w_{ki}^{L}\geq \epsilon, i\in N. 
        \end{split}
    \end{align}
$$


In [5]:
solution = solveCrispAHPLP(A₁)
V = solution.V
display(L"V = %$(intervalVectorLaTeXString(V))")

L"$V = \begin{pmatrix} \left[ 0.5, 0.5 \right] \\ \left[ 0.25, 0.25 \right] \\ \left[ 0.125, 0.167 \right] \\ \left[ 0.083, 0.125 \right] \end{pmatrix}$"

In [6]:
d = solution.optimalValue

0.08333333333333326

### 質問
perfect incorporation approach

$$
    \begin{align}
        \begin{split}
            \text{minimize} ~~ & ~~ \sum_{i\in N} (w_{i}^{U} - w_{i}^{L}) \\
            \text{subject to} ~~ & ~~ \sum_{i\in N} (w_{ki}^{U} - w_{ki}^{L}) \leq \hat{d_{k}} , k\in M \\
            ~~ & ~~ w_{ki}^{L}\leq a_{kij}w_{kj}^{R},\, a_{kij}w_{kj}^{L}\leq w_{ki}^{U},\, i,j,\in N(i<j) , k\in M\\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{R} + w_{ki}^{L}\geq 1, k\in M, i\in N, \\
            ~~ & ~~ \sum_{j\in N\setminus i} w_{kj}^{L} + w_{ki}^{U}\leq 1, k\in M, i\in N, \\
            ~~ & ~~ \epsilon \leq w_{i}^{L}\leq w_{ki}^{L}\leq w_{ki}^{U} \leq w_{i}^{U}, k\in M, i\in N.
        \end{split}
    
    \end{align}
$$

重みをt倍するときに、kごとに$t_{k}$みたいな感じにするのか？